## dataset preparation

In [3]:
import cv2
import numpy as np
from keras.models import Model
from keras.models import load_model
import pandas as pd
from tqdm import tqdm
import tensorflow as tf

## age or gender through caffe model

In [4]:
import cv2
import imutils
import time
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping 

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

In [5]:
def initialize_caffe_models():
	
	age_net = cv2.dnn.readNetFromCaffe(
		'data/deploy_age.prototxt', 
		'data/age_net.caffemodel')

	gender_net = cv2.dnn.readNetFromCaffe(
		'data/deploy_gender.prototxt', 
		'data/gender_net.caffemodel')

	return(age_net, gender_net)

In [6]:
def age_gender_of_face(age_net,gender_net,file):
    # Get Face 
    face_img = cv2.imread(file,cv2.IMREAD_COLOR)
#     plt.imshow(face_img)
    blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

    #Predict Gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_preds[0].argmax()
#     print("Gender : " + gender)

    #Predict Age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_preds[0].argmax()
#     print("Age Range: " + age)
    return age , gender

In [7]:
age_net, gender_net = initialize_caffe_models()
file = '../face_generator/celebA/img_align_celeba/img_align_celeba/000001.jpg'
age_gender_of_face(age_net,gender_net,file)

(2, 1)

### End

## race     ethnicity = ["Asian",	"Black", "Hispanic","Indian",	"White"]

In [8]:
race_model = load_model('run20-model.h5')
race_model.load_weights('run20-model.h5')

In [13]:
def get_data_from_local(file,w,h):
    face = cv2.imread(file,cv2.IMREAD_COLOR)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face =cv2.resize(face, (w,h))
    face = np.asarray(face)
    face = face.astype('float32')
    face /= 255
    return np.array(face)

In [11]:
data1 = pd.read_csv('celebA/list_attr_celeba.csv')
data1 = data1.sample(frac = 1)[:200]
data1 = data1.replace(-1,0)
data1.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
17151,017152.jpg,0,1,1,0,0,1,1,0,1,...,0,0,0,1,0,0,1,1,0,1
160671,160672.jpg,0,1,1,0,0,1,0,0,0,...,0,1,1,0,1,0,1,1,0,0
199855,199856.jpg,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,1,1,0,0
38729,038730.jpg,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
82199,082200.jpg,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1


In [16]:
data = {
    'age':[],
    'gender':[],
    'race':[]
}
# race_model = load_model('model_3/final_cnn_race_model_checkpoint.tf')
for image in tqdm(data1['image_id']):
    temp = image
    file = 'celebA/img_align_celeba/img_align_celeba/'+image
    image = get_data_from_local(file,200,200)
    image = np.expand_dims(image, axis=0)
    age , gender = age_gender_of_face(age_net,gender_net,file)
    data['age'].append(age)
    data['gender'].append(gender)
    data['race'].append(race_model.predict(np.expand_dims(get_data_from_local(file,64,64),axis=0)).argmax())

100%|██████████| 200/200 [00:43<00:00,  4.59it/s]


In [17]:
data1.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [18]:
data1['age'] = data['age']
data1['gender'] = data['gender']
data1['race'] = data['race'] 

In [19]:
data2 = pd.get_dummies(data=data1,columns=['age','gender','race'])

In [20]:
print(data2.columns)
len(data2.columns)

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young', 'age_0', 'age_1', 'age_2', 'age_3', 'age_4',
       'age_5', 'age_6', 'age_7', 'gender_0', 'gender_1', 'race_0', 'race_1',
       'race_2', 'race_3', 'race_4'],
      dtype='object')


56

In [21]:
len(data2.columns)

56

In [22]:
data2.to_csv('decode_datasest.csv')

# prediction

In [ ]:
def training_dataset_(
    file_path,
    image_per_batch,
    start_row,
    ):
    df = pd.read_csv(file_path)[start_row:start_row+image_per_batch]
    df = df.drop('Unnamed: 0',axis=1)

    y = get_data_from_local(df)
    x = df.drop('image_id',axis=1)
    
    x = x.astype('float64')
        
    return x.values , y

In [ ]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['image_id']):
        file = 'celebA/img_align_celeba/img_align_celeba/' + file
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face =cv2.resize(face, (150,150))
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

In [ ]:
c_a_e = load_model('decoder_cnn_50.tf/')
c_a_e.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
c_a_e.summary()

In [ ]:
def training(
    start_row,
    number_of_labels,
    file_path,
    image_per_batch,
    max_limit,
    path_of_model
):

    checkpoint = ModelCheckpoint(filepath=path_of_model,
                             monitor='val_accuracy',
                             save_format='tf',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )
    early_stopping_monitor = EarlyStopping(patience=4,verbose=1) 
    
    
    for x in range(start_row,max_limit,image_per_batch):
        
        print()
        print('**********{} -- {} out of {}************'.format(x,x+image_per_batch,max_limit))
        print()
    
        X_train , y_train   =   training_dataset_(
                        file_path = file_path,
                        start_row = x,
                        image_per_batch = image_per_batch
        )
                                    
        F_mnist = c_a_e.fit(X_train,y_train,
                            shuffle=True,
                            epochs=30,
                            validation_split=0.2,
                            batch_size=64,
                            callbacks=[checkpoint,early_stopping_monitor]
       )


In [ ]:
training(
    start_row = 0,
    number_of_labels = 3,
    file_path = 'decode_datasest.csv',
    image_per_batch = 500,
    max_limit = 500,
    path_of_model = 'final_dec_cnn_checkpoint.tf'
)

In [ ]:
# 